In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [0]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

def loadbatch(batchname):
    folder = 'D:\\Games\\cifar-10-batches-py'
    batch = unpickle(folder+"/"+batchname)
    return batch

def loadlabelnames():
    folder = 'D:\\Games\\cifar-10-batches-py'
    meta = unpickle(folder+"/"+'batches.meta')
    return meta[b'label_names']

def visualise(data, index):
    # images will be quite fuzzy looking, because they are low res (32x32).

    picture = data[index]
    # Initially, the data is a 1D array of 3072 pixels; reshape it to a 3D array of 3x32x32 pixels
    picture.shape = (3,32,32) 
    picture = picture.transpose([1, 2, 0])
    plt.imshow(picture)
    plt.show()

In [0]:
batch2 = loadbatch('data_batch_2')
print("Number of items in the batch is", len(batch2))
print('All keys in the batch:', batch2.keys())

data = batch2[b'data']
labels = batch2[b'labels']
print ("size of data in this batch:", len(data), ", size of labels:", len(labels))
print(data.shape)

names = loadlabelnames()

In [0]:
i1 = []
for i,x in enumerate(labels):
    if names[x] == b'truck' or names[x] == b'automobile':
        i1.append(i)

data_final = []
labels_final = []

for i in i1:
    data_final.append(np.array(data[i]).reshape(3,1024).T)
    labels_final.append(labels[i])
    
len(data_final)

1992

In [0]:
data_train = []

for i in range(len(data_final)):
    for l in data_final[i]:
        data_train.append(l[0])
    
data_train = np.array(data_train).reshape(len(data_final),1024)
# data_train.shape

In [0]:
for i in range(len(labels_final)):
    if names[labels_final[i]] == b'truck':
        labels_final[i] = 0

for i in range(len(labels_final)):
    if names[labels_final[i]] == b'airplane':
        names[labels_final[i]] = b'truck'
        
print(names[labels_final[0]])
print(names[labels_final[2]])

b'automobile'
b'truck'


In [0]:
x = pd.DataFrame(data_train)
y = pd.DataFrame(labels_final)

X_train = x.sample(frac=0.8, random_state=200)
X_test = x.drop(X_train.index)

X_train = np.array(X_train)
X_test = np.array(X_test)

y_train = y.sample(frac=0.8, random_state=200)
y_test = y.drop(y_train.index)

y_train = np.array(y_train)
y_test = np.array(y_test)

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(1594, 1024)
(1594, 1)
(398, 1024)
(398, 1)


In [0]:
def sigmoid(x):
    return 1/(1+np.exp(-x))

def sigmoidPrime(x):
    return x * (1 - x)

input_nodes = 1024
hidden_nodes = 1024
output_nodes = 1

np.random.seed(1)
#Weights
w1 = np.random.randn(input_nodes, hidden_nodes)  - 1 
w2 = np.random.randn(hidden_nodes, output_nodes) - 1
bias1 = np.random.randn(1,hidden_nodes)
bias2 = np.random.randn(1,output_nodes)

learning_rate = 0.00005

loss = []
m = len(X_train)
loss_final = []

for i in range(10000):  
    layer1 = X_train
    layer2 = sigmoid(np.dot(layer1,w1) + bias1)
    layer3 = sigmoid(np.dot(layer2,w2) + bias2)

    # Loss Calculation

    layer3_loss = np.sqrt((y_train - layer3)**2)
    layer3_delta = layer3_loss * sigmoid(layer3)

    layer2_loss = (np.dot(layer3_delta, w2.T))
    layer2_delta = layer2_loss * sigmoid(layer2)

    w2 = w2 + layer2.T.dot(layer3_delta)*learning_rate
    w1 = w1 + layer1.T.dot(layer2_delta)*learning_rate
    bias1 = bias1 + np.sum(layer2_delta, axis=0, keepdims = True)*learning_rate
    bias2 = bias2 + np.sum(layer3_delta, axis=0, keepdims = True)*learning_rate

In [0]:
y_test

In [0]:
layer1 = X_test
layer2 = sigmoid(np.dot(layer1, w1))
layer3 = sigmoid(np.dot(layer2, w2))
print(layer3)

cv = [np.round(i) for i in layer3]
cc = 0
for c in range(len(cv)):
    if int(cv[c]) == int(y_test[c]):
        cc += 1
accuracy = (cc/len(cv)) * 100

print("Validation Accuracy " + str(round(accuracy,2)) + "%")